In [1]:
# Import packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html

# Graph 2: Global Expansion of Renewables to Achieve the 1.5C Goal

In [2]:
# Map each energy type to specified color
col_map = {'PV':'#F0B40E','Solar thermal power plants':'#EC5009','Geothermal Power':'#783196',
           'Wind':'#3B8090','Ocean energy':'#1D373E','Hydro Power':'#343F9C','Biomass Power':'#275725',
           'Hydrogen Power':'#D70009','Nuclear':'#B7B3AB','Gas':'#857C77','Coal & Lignite':'#433C3E',
           'Oil & Diesel':'#141414','Solar Heat':'#EC5009','Biomass':'#275725','Geothermal':'#783196',
           'H Process':'#D70009','RE district heat':'#883C15','H Fuel':'#D055A2','Synfuels':'#6883A0',
           'Biofuels':'#7FCBA5'}

# Read Data Source
renew = pd.read_csv('Data/Renewables.csv')

# Finds sum of each energy categories demand for each year
add_total = renew.groupby(['Energy Category', 'Year']).agg({'Final Energy Demand': 'sum'})

for y in range(2015,2055, 5):
    for c in ['Clean Fuels', 'Clean Power', 'Fossil Fuels', 'Clean Heat']:
        insert_value = add_total.loc[c, y].values # Finds the total category demand to add for year
        index = list(renew.loc[(renew['Energy Category'] == c) & (renew['Year'] == y)].index)
        renew.loc[index, 'Total'] = insert_value # Adds the total category demand to the original data frame

# Calculates the total percentage demand for a carrier in the energy category
renew['Total Percentage'] = round(renew['Final Energy Demand'] / renew['Total'] * 100, 2)

# Loops through the color map and adds acolor column to the dataframe 
for single in list(col_map.keys()):
    renew.loc[renew['Energy Type'] == single, 'Color'] = col_map[single]

# Subsets original data frame by energy category and year
cleanpower = renew.loc[(renew['Energy Category'] == 'Clean Power') & (renew['Year'] == 2050), 'CumSumEnergy']
fossil = renew.loc[(renew['Energy Category'] == 'Fossil Fuels') & (renew['Year'] == 2050), 'CumSumEnergy']
cleanheat = renew.loc[(renew['Energy Category'] == 'Clean Heat') & (renew['Year'] == 2050), 'CumSumEnergy']
cleanfuels = renew.loc[(renew['Energy Category'] == 'Clean Fuels') & (renew['Year'] == 2050), 'CumSumEnergy']

# Create figure
fig = go.Figure()

# Stacked area plot
for t in renew['Energy Type'].unique():
    plot_data = renew.loc[renew['Energy Type']==t,]
    # Adds the cumulative sum of trace to create a stacked area plot
    fig.add_trace(go.Scatter(x = plot_data['Year'],
                             y = plot_data['Final Energy Demand'],
                             mode = 'none',
                             fill = 'tonexty', fillcolor = col_map[t],
                             stackgroup = 'one',
                             name = t,
                             hoverinfo = 'none'))
    
# Add vertical lines every five years
for y in range(2020,2050,5):
    fig.add_shape(type = 'line',
                  x0 = y, y0 = 0,
                  x1 = y, y1 = renew.loc[renew['Year']==y,'Final Energy Demand'].sum(),
                  line = dict(color = 'white', width = 2, dash = 'dot'))   

# Creates the Clean fuel brackets at right edge of plot
fig.add_shape(type = 'line',
              x0 = 0.99, y0 = cleanheat.max(),
              x1 = 0.99, y1 = cleanfuels.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))
fig.add_shape(type = 'line',
              x0 = 0.98, y0 = cleanfuels.max(),
              x1 = 0.99, y1 = cleanfuels.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))
fig.add_shape(type = 'line',
              x0 = 0.98, y0 = cleanheat.max(),
              x1 = 0.99, y1 = cleanheat.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))

# Creates the Clean Heat brackets at right edge of plot
fig.add_shape(type = 'line',
              x0 = 0.99, y0 = fossil.max(),
              x1 = 0.99, y1 = cleanheat.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))
fig.add_shape(type = 'line',
              x0 = 0.98, y0 = cleanheat.max(),
              x1 = 0.99, y1 = cleanheat.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))
fig.add_shape(type = 'line',
              x0 = 0.98, y0 = fossil.max(),
              x1 = 0.99, y1 = fossil.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))

# Creates the Clean Power brackets at right edge of plot
fig.add_shape(type = 'line',
              x0 = 0.99, y0 = 0,
              x1 = 0.99, y1 = cleanpower.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))
fig.add_shape(type = 'line',
              x0 = 0.98, y0 = cleanpower.max(),
              x1 = 0.99, y1 = cleanpower.max(),
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))
fig.add_shape(type = 'line',
              x0 = 0.98, y0 = 0,
              x1 = 0.99, y1 = 0,
              xref = 'paper', yref = 'y',
              line = dict(color = 'grey', width = 2, dash = 'solid'))

# Subsets data to avoid hover text for energy categories with less than 1 KJ usage 
renew = renew.loc[renew['Final Energy Demand'] > 1]
# Adds the hover text region, creates transparent circles at the intersection of each area
# Hover text includes information about the dataset
fig.add_trace(go.Scatter(x = renew['Year'],
                         y = renew['CumSumEnergy'],
                         marker = dict(color = 'black', size = 5, opacity = 0),
                         name = '',
                         mode = 'markers',
                         hoverlabel = dict(align = 'left'),
                         customdata = renew[['Final Energy Demand', 'Total Percentage', 'Energy Category',
                                            'Energy Type', 'Total', 'Color']],
                         hovertemplate = '<br> Year: <b>%{x}</b>' + 
                                         '<br>Energy Carrier: <b><span style="color:%{customdata[5]}">%{customdata[3]}</span></b>'+
                                         '<br>Carrier Demand: <b>%{customdata[0]:,.0f} PJ </b>'+
                                         '<br>Energy Category: <b>%{customdata[2]} </b>' + 
                                         '<br>Category Demand: <b>%{customdata[4]:,.0f} PJ </b>' + 
                                         '<br>Carrier Percentage: <b>%{customdata[1]:,.0f}% </b>'))


# Positions the Fossil Fuel Lable
fig.add_annotation(x = 2017.5, y = (renew.loc[(renew['Energy Category'] == 'Fossil Fuels') & (renew['Year'] == 2015),'CumSumEnergy'].max() \
                                    + renew.loc[(renew['Energy Category'] == 'Clean Power') & (renew['Year'] == 2015),'CumSumEnergy'].max()) / 2,
                   xref = 'x', yref = 'y',
                   axref = 'x', ayref = 'y',
                   text = 'Fossil Fuels', align = 'left',
                   font = dict(family = 'Arial', size = 14, color = 'white'), showarrow = False)

# Positions the Clean Fuel Label
fig.add_annotation(x = 1.13, y = (cleanfuels.max() + cleanheat.max()) / 2,
                   xref = 'paper', yref = 'y',
                   axref = 'x', ayref = 'y',
                   text = 'Clean Fuels', align = 'left',
                   font = dict(family = 'Arial', size = 14, color = 'black'), showarrow = False)

# Positions the Clean Heat Label
fig.add_annotation(x = 1.13, y = (cleanheat.max() + fossil.max()) / 2,
                   xref = 'paper', yref = 'y',
                   axref = 'x', ayref = 'y',
                   text = 'Clean Heat', align = 'left',
                   font = dict(family = 'Arial', size = 14, color = 'black'), showarrow = False)

# Positions the Clean Power Label
fig.add_annotation(x = 1.13, y = cleanpower.max() / 2,
                   xref = 'paper', yref = 'y',
                   axref = 'x', ayref = 'y',
                   text = 'Clean Power', align = 'left',
                   font = dict(family = 'Arial', size = 14, color = 'black'), showarrow = False)


# Updates the layout of the plot
fig.update_layout({'plot_bgcolor':'rgba(0, 0, 0, 0)', 'paper_bgcolor':'rgba(0, 0, 0, 0)'},
                  width = 800, height = 800,
                  margin_pad = 5,
                  font_family = 'Calibri (body)',
                  title = {'text':'<b>Global Expansion of Renewables to Achieve the 1.5C Goal</b>',
                           'font_size':20,
                           'font_color':'black',
                           'y':0.9,
                           'yanchor':'top'},
                  showlegend = False,
                  xaxis = dict(range = [2015,2051],
                               tickfont = dict(color = 'black', size = 14)),
                  yaxis = dict(range = [0,400000],
                               tickmode = 'array',
                               tickvals = [50000,100000,150000,200000,250000,300000,350000],
                               ticktext = ['50,000','100,000','150,000','200,000','250,000','300,000','350,000'],
                               tickfont = dict(color = 'black', size = 14),
                               title_standoff = 25,
                               title = {'text':'<b>Final Energy Demand (PJ)</b>',
                                        'font_size':16,
                                        'font_color':'black'}),
                  hoverlabel = dict(bgcolor = 'white', font_size = 15))

# Display the figure
#fig.show()

# To pass the figure into a Dash app, comment out fig.show() above, then uncomment and run the following lines
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Graph(figure = fig)
])

if __name__ == '__main__':
    app.run_server(mode = 'inline', debug = False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jan/2021 16:39:20] "GET /_alive_b1e21be2-6a7d-4852-bd98-29bbe4de8ea9 HTTP/1.1" 200 -


127.0.0.1 - - [05/Jan/2021 16:39:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:39:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:39:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:39:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:39:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:39:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:49:18] "GET /_shutdown_dd0ac356-d233-45a4-af2f-95b28be40fd7 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:49:18] "GET /_alive_dd0ac356-d233-45a4-af2f-95b28be40fd7 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:49:18] "GET /_alive_dd0ac356-d233-45a4-af2f-95b28be40fd7 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:49:18] "GET /_alive_dd0ac356-d233-45a4-af2f-95b28be40fd7 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:49:18] "GET /_alive_dd0ac356-d233-45a4-af2f-95b28be40fd7 HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:49:18] "GET /_alive_dd0ac356-d23